# 使用神经网络来训练获得Embedding矩阵

In [2]:
# 查看tensorflow版本
import tensorflow as tf
print(tf.__version__)

2.3.0


In [3]:
# 导入tensorflow的数据集
import tensorflow_datasets as tfds
imdb,info = tfds.load("imdb_reviews",with_info=True,as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDV4VQK/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDV4VQK/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDV4VQK/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
# 查看导入的数据集
print(type(imdb))
print(imdb)
print(info)

<class 'dict'>
{'test': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>, 'train': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>, 'unsupervised': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>}
tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('t

In [5]:
# 对导入数据进行处理(存储到对应的列表中)
import numpy as np

train_data, test_data = imdb['train'],imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf-8'))
  training_labels.append(l.numpy())

print("------")
print(len(training_sentences))

for s,l in test_data:
  testing_sentences.append(s.numpy().decode("utf-8"))
  testing_labels.append(l.numpy())

print(testing_sentences[0])
print(testing_sentences[1])

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)
print(testing_labels_final)

------
25000
There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come.
A blackly comic tale of a down-trodden priest, 

In [6]:
# 先定义好对应的参数，再对training_sentences进行分词处理
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 选择频率最高的前vocab_size个单词
tokenizer = Tokenizer(num_words = vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)

padded = pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)

# 并不会改变training_sentences
print("想知道training_sentences有没有改变：",training_sentences[0:2])

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

想知道training_sentences有没有改变： ["This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", 'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbi

In [7]:
# 将词索引改变成数索引
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

# 将数字转换为文本
def decode_review(text):
  # 获得key=i的val,如果不存在返回?
  return ''.join([reverse_word_index.get(i,"?") for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

???????????????????????????????thisisthekindoffilmforasnowysundayafternoonwhentherestoftheworldcangoaheadwithitsownbusinessasyou<OOV>intoabigarmchairand<OOV>foracoupleofhourswonderfulperformancesfromcherandnicolascageasalwaysgentlyrowtheplotalongthereareno<OOV>tocrossnodangerouswatersjustawarmandwitty<OOV>throughnewyorklifeatitsbestafamilyfilmineverysenseandonethatdeservesthepraiseitreceived
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [8]:
# 构建一个神经网络模型
model = tf.keras.Sequential([
  # vocab:词汇表维度
  # embedding_dim:嵌入词空间的维度
  # input_length：输入语句的长度
  tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(6,activation='relu'),
  tf.keras.layers.Dense(1,activation='sigmoid')
])

# 编译模型
model.compile(loss='binary_crossentropy',optimizer = 'adam',metrics=['accuracy'])

# 查看模型结构
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [9]:
# 训练模型
num_epochs = 10
model.fit(padded,
    training_labels_final,
    epochs=num_epochs,
    validation_data=(testing_padded,testing_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 7ms/step - loss: 0.4984 - accuracy: 0.7397 - val_loss: 0.3454 - val_accuracy: 0.8486
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2426 - accuracy: 0.9064 - val_loss: 0.3583 - val_accuracy: 0.8464
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0993 - accuracy: 0.9734 - val_loss: 0.4353 - val_accuracy: 0.8326
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0254 - accuracy: 0.9967 - val_loss: 0.5301 - val_accuracy: 0.8282
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0070 - accuracy: 0.9994 - val_loss: 0.5938 - val_accuracy: 0.8268
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0022 - accuracy: 0.9999 - val_loss: 0.6393 - val_accuracy: 0.8288
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 9.1030e-04 - accuracy: 1.0000 - val_loss: 0.6773 - val_accuracy: 0.8309
Ep

In [10]:
# 查看模型第一层和对应权重
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)
print(weights)

(10000, 16)
[[-0.01879507  0.00729668 -0.04908711 ...  0.00048328  0.007699
  -0.04019675]
 [-0.08698982  0.01082476 -0.09326092 ... -0.04265146  0.01316135
  -0.07230072]
 [-0.08767401 -0.00989071 -0.11094046 ... -0.0527577  -0.00163758
  -0.05354297]
 ...
 [-0.05521015  0.06092997 -0.04866304 ...  0.00733782 -0.03679712
   0.14384678]
 [-0.10652543  0.10175183  0.06349761 ...  0.12936865 -0.08399879
   0.0256535 ]
 [-0.05182629  0.08749562 -0.02704544 ...  0.06271378 -0.00168708
  -0.04915047]]


In [12]:
# 将其embeddings矩阵(第一层的权重矩阵)和词汇表存储到文件上
import io
out_v = io.open('vecs.tsv','w',encoding='utf-8')
out_m = io.open('meta.tsv','w',encoding='utf-8')

for word_num in range(1,vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word+'\n')
  out_v.write('\t'.join([str(x) for x in embeddings]) + '\n')

out_v.close()
out_m.close()

In [13]:
# 使用colab代码将文件下载下来
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>